# 1. Importazione delle librerie

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from torchvision.datasets import CIFAR10
from torchvision import transforms
import matplotlib.pyplot as plt

# 2. Caricamento del dataset CIFAR-10

In [3]:
# Trasformazione per convertire le immagini in tensori
transform = transforms.Compose([transforms.ToTensor()])

# Download del dataset CIFAR10 (train e test)
cifar_train = CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_test = CIFAR10(root='./data', train=False, download=True, transform=transform)

# Estrazione immagini e label in formato NumPy, con flattening (vettorizzazione delle immagini 32x32x3 → 3072)
X_train = np.array([np.array(img).flatten() for img, _ in cifar_train])
y_train = np.array([label for _, label in cifar_train])
X_test = np.array([np.array(img).flatten() for img, _ in cifar_test])
y_test = np.array([label for _, label in cifar_test])

100%|██████████| 170M/170M [00:03<00:00, 52.1MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# 3. Sottocampionamento per ridurre la complessità computazionale

In [4]:
from sklearn.utils import resample

# Funzione per sottocampionare: seleziona N immagini per classe in modo casuale
def undersample_all_classes(X, y, samples_per_class=200):
    unique_classes = np.unique(y)
    X_resampled = []
    y_resampled = []
    for cls in unique_classes:
        # Filtra solo le immagini della classe corrente
        X_cls = X[y == cls]
        y_cls = y[y == cls]

        # Sottocampiona n_samples immagini in modo casuale
        X_down, y_down = resample(X_cls, y_cls, n_samples=samples_per_class, random_state=42)
        X_resampled.append(X_down)
        y_resampled.append(y_down)

    # Unisce tutte le classi sottocampionate in un unico array
    X_final = np.vstack(X_resampled)
    y_final = np.hstack(y_resampled)
    return X_final, y_final

# Applica il sottocampionamento: 200 immagini per ciascuna classe = 2000 totali
X_train_resampled, y_train_resampled = undersample_all_classes(X_train, y_train, samples_per_class=200)

# 4. Normalizzazione e riduzione dimensionale (PCA)

In [5]:
# Normalizzazione dei dati per portare tutte le feature su scala simile (media 0, dev std 1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# PCA per ridurre la dimensionalità del dataset mantenendo la maggior parte della varianza
pca = PCA(n_components=30)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# 5. Funzione per la Grid Search e training dei modelli

In [6]:
# Funzione che applica GridSearchCV per trovare i migliori iperparametri
def train_model(model, param_grid, X_train, y_train):
    grid = GridSearchCV(model, param_grid, cv=3)
    grid.fit(X_train, y_train)
    return grid.best_estimator_, grid.best_params_

# 6. Addestramento dei modelli con selezione degli iperparametri

In [ ]:
# Regressione Logistica
log_reg_params = {'C': [0.1, 1, 10]}
log_reg_model, log_reg_best_params = train_model(LogisticRegression(max_iter=1000), log_reg_params, X_train_pca, y_train_resampled)
print(f"Logistic Regression - Best Params: {log_reg_best_params}")

# k-Nearest Neighbors
knn_params = {'n_neighbors': [3, 5, 7]}
knn_model, knn_best_params = train_model(KNeighborsClassifier(), knn_params, X_train_pca, y_train_resampled)
print(f"k-NN - Best Params: {knn_best_params}")

# Support Vector Machine
svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear']}
svm_model, svm_best_params = train_model(SVC(), svm_params, X_train_pca, y_train_resampled)
print(f"SVM - Best Params: {svm_best_params}")

# Decision Tree
dt_params = {'max_depth': [10, 20, 30, None]}
dt_model, dt_best_params = train_model(DecisionTreeClassifier(), dt_params, X_train_pca, y_train_resampled)
print(f"Decision Tree - Best Params: {dt_best_params}")

Logistic Regression - Best Params: {'C': 10}
k-NN - Best Params: {'n_neighbors': 5}


# 7. Valutazione dei modelli e visualizzazione delle confusion matrix

In [ ]:
# Valutazione dei modelli su test set reale
models = [log_reg_model, knn_model, svm_model, dt_model]
model_names = ["Logistic Regression", "k-NN", "SVM", "Decision Tree"]

for i, model in enumerate(models):
    y_pred = model.predict(X_test_pca)
    print(f"\nResults for {model_names[i]}:")
    print(classification_report(y_test, y_pred))

    # Plot confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    plt.title(f"Confusion Matrix for {model_names[i]}")
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.colorbar()
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.xticks(np.arange(10), np.arange(10))
    plt.yticks(np.arange(10), np.arange(10))
    plt.show()

# 8. Selezione del miglior modello in base all’accuratezza

In [ ]:
# Selezione automatica del modello con la maggiore accuratezza
best_model = max(
    zip(models, model_names),
    key=lambda x: classification_report(y_test, x[0].predict(X_test_pca), output_dict=True)['accuracy']
)
print(f"\nBest Model: {best_model[1]}")